Installing the required libraries

In [1]:
!pip install pandas
!pip install sentence-transformers
!pip install setfit
!pip install scikit-learn
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

Importing the necessary libraries

In [2]:
import pandas as pd
import json
import os
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers.losses import CosineSimilarityLoss
from datasets import Dataset
from sklearn.metrics import classification_report
from collections import defaultdict

In [3]:
BASE_MODEL = "BAAI/bge-base-en-v1.5"
RANDOM_SEED = 42
OUTPUT_PATH = 'output'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_dir = "/content/drive/MyDrive/multilabel_classification"
df = pd.read_csv(os.path.join(data_dir, "data.csv"))

In [6]:
df["text"] = df["id"] + " " + df["readme"]
df.drop(columns=["id", "readme"], axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
target_list = list([col for col in df.columns if col!="text"])
target_list

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['Dependency management',
 'Support',
 'Testing',
 'Reporting',
 'Mobile CI',
 'Community',
 'Security',
 'Chat',
 'IDEs',
 'Monitoring',
 'Code review',
 'Code quality',
 'Publishing',
 'Project management',
 'API management',
 'Open Source management',
 'Mobile',
 'Code search',
 'GitHub Sponsors',
 'Continuous integration',
 'Deployment',
 'Container CI',
 'Localization',
 'Utilities',
 'Desktop tools',
 'AI Assisted',
 'Time tracking',
 'Code Scanning Ready',
 'Backup Utilities',
 'Learning']

In [8]:
def prepare_multilabel_label_col(row, targets):
    values = [row[target] for target in targets]

    return values


In [9]:
from sklearn.model_selection import train_test_split

df["label"] = df.apply(lambda x: prepare_multilabel_label_col(x, target_list), axis=1)
df_copy = df.copy()

df_copy.drop(columns=target_list, axis=1, inplace=True)
train_df, test_df = train_test_split(df_copy, random_state=42, test_size=0.20, shuffle=True)

train_df, validation_df = train_test_split(train_df, random_state=42, test_size=0.10, shuffle=True)

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


In [10]:
train_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,label
564,varunsridharan/action-wp-org-deploy b'# WordPr...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1091,timkrase/phpunit-coverage-badge b'# phpunit-co...,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
822,RafikFarhad/clang-format-github-action b'# Cla...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ..."
1087,MilesChou/composer-action b'# Docker and GitHu...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
646,alexellis/upload-assets b'# upload-assets\n\nG...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [11]:
train_set = Dataset.from_pandas(train_df)
test_set = Dataset.from_pandas(test_df)
validation_set = Dataset.from_pandas(validation_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    multi_target_strategy="one-vs-rest",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [13]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_set,
    eval_dataset=validation_set,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=4,
    num_epochs=1,
    num_iterations=20,
)
trainer.train()
y_pred = trainer.model.predict(test_set["text"])
metrics = classification_report(test_set["label"], y_pred, digits=4, output_dict=True)


<ipython-input-13-59c005f9a2e5>:1: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/873 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 34920
  Batch size = 4
  Num epochs = 1
  Total optimization steps = 8730


Step,Training Loss


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [17]:
def get_prediction_for_each_label(predictions, target_list):
    values = {}
    for i, target in enumerate(target_list):
        values["{0}_prediction".format(target)] = predictions[i]

    return values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
def get_true_value_for_each_label(label, target_list):
    values = {}
    for i, target in enumerate(target_list):
        values["{0}_label".format(target)] = label[i]

    return values

In [22]:
updated_test_data = []

count = 0
for i, row in test_df.iterrows():
    predictions = get_prediction_for_each_label(row["label"], target_list)

    row_labels = get_true_value_for_each_label(y_pred[count], target_list)

    for target in target_list:
        row["{0}_prediction".format(target)] = predictions["{0}_prediction".format(target)]
        row["{0}_label".format(target)] = row_labels["{0}_label".format(target)]

    updated_test_data.append(row)
    count += 1

test_data = pd.DataFrame(updated_test_data)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
import json
from sklearn.metrics import accuracy_score

accuracy_data = {}

accuracy_scores = []
for target in target_list:
    preds = test_data["{0}_prediction".format(target)].to_list()
    labels = test_data["{0}_label".format(target)].to_list()

    accuracy = accuracy_score(labels, preds)
    accuracy_data[target] = accuracy
    accuracy_scores.append(accuracy)

print("Accuracy for each label:")
accuracy_data = json.dumps(accuracy_data, indent = 4)
print(accuracy_data)
print("Average accuracy score: ", sum(accuracy_scores)/len(accuracy_scores)*100)


Accuracy for each label:
{
    "Dependency management": 0.9259259259259259,
    "Support": 0.9958847736625515,
    "Testing": 0.9259259259259259,
    "Reporting": 0.9711934156378601,
    "Mobile CI": 0.9835390946502057,
    "Community": 0.9835390946502057,
    "Security": 0.9753086419753086,
    "Chat": 0.9917695473251029,
    "IDEs": 1.0,
    "Monitoring": 0.9917695473251029,
    "Code review": 0.8683127572016461,
    "Code quality": 0.8765432098765432,
    "Publishing": 0.9176954732510288,
    "Project management": 0.9218106995884774,
    "API management": 0.9876543209876543,
    "Open Source management": 0.9259259259259259,
    "Mobile": 0.9876543209876543,
    "Code search": 0.9917695473251029,
    "GitHub Sponsors": 1.0,
    "Continuous integration": 0.6666666666666666,
    "Deployment": 0.8600823045267489,
    "Container CI": 0.9382716049382716,
    "Localization": 1.0,
    "Utilities": 0.6625514403292181,
    "Desktop tools": 0.9958847736625515,
    "AI Assisted": 1.0,
    "Time